# Chatbot 

## Introduction

### Imports

In [1]:
import nltk
from nltk.stem.lancaster import LancasterStemmer
import numpy as np
import tflearn
import tensorflow as tf
import random 
import json

nltk.download('punkt')

W0214 14:10:59.599688 140735779611520 deprecation.py:323] From /Users/spags/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/tensorflow/python/compat/v2_compat.py:96: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
Instructions for updating:
non-resource variables are not supported in the long term
[nltk_data] Downloading package punkt to /Users/spags/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

## Obtain

### Importing json

In [2]:
# Open json

with open('intents.json') as file:
    data = json.load(file)
    
data

{'intents': [{'tag': 'greeting',
   'patterns': ['Hi',
    'Hello',
    'How are you',
    "What's up",
    'Howdy',
    'Good day',
    'Is anyone there?',
    'yo',
    'Hey'],
   'responses': ['Hello!',
    'Good to see you again!',
    'Hi there, how can I help?',
    'Howdy partner!',
    'Hey there!'],
   'context_set': ''},
  {'tag': 'goodbye',
   'patterns': ['goodbye',
    'bye',
    'cya',
    'have a nice day',
    'have a good day',
    'peace'],
   'responses': ['Bye for now!',
    'Goodbye, friend!',
    'Talk to you later!',
    'Adios, muchacho!'],
   'context_set': ''}]}

## Scrub

### Preprocessing text data

In [3]:
# loop through json and pull out patterns

words = []  # list of words
docs_x = [] # word sequences
docs_y = [] # y is the tag as well, for modeling purposes
labels = []  # tag

for intent in data['intents']:
    for pattern in intent['patterns']:  # stemming will take each word in a pattern and bring it down to root word
        word = nltk.word_tokenize(pattern)
        words.extend(word)  # use extend rather than append because it's already a list (faster)
        docs_x.append(word)
        docs_y.append(intent['tag'])
    if intent['tag'] not in labels:
        labels.append(intent['tag'])

In [4]:
# Stemming out the words list

stemmer = LancasterStemmer()

words = [stemmer.stem(w.lower()) for w in words if w not in '?']  # removes question mark
words = sorted(list(set(words)))    # using set to remove duplicates and then converting back to list...cool idea

In [5]:
# Sorting labels for funsies

labels = sorted(labels)

In [6]:
# Create a bag of words to train the model (one hot endcoded)

training = []
output = []

out_empty = [0 for _ in range(len(labels))]

for x, doc in enumerate(docs_x):  # actually putting the bag of words together here
    bag = []
    word_list = [stemmer.stem(w) for w in doc]
    
    for w in words:
        if w in word_list:
            bag.append(1)
        else:
            bag.append(0)
    # Generating Output        
    output_row = out_empty[:]
    output_row[labels.index(docs_y[x])] = 1
    
    # filling training & output
    training.append(bag)
    output.append(output_row)

In [7]:
# Changing trainign and output to numpy arrays

training = np.array(training)
output = np.array(output)

## Modeling

In [8]:
len(training[0])

23

In [9]:
len(output[0])

2

In [10]:
# Setting up a neural network with tflearn...new one for me

net = tflearn.input_data(shape = [None, len(training[0])]) # input layer?
net = tflearn.fully_connected(net, 8)  # adds fully connected hidden layer to the network with 8 neurons
net = tflearn.fully_connected(net, 8) # adds second hidden layer
net = tflearn.fully_connected(net, len(output[0]), activation = 'softmax') # output layer
net = tflearn.regression(net) # adds a regression layer?

# compile the model
model = tflearn.DNN(net)

# fit the model (n_epoch in tflearn, show_metric gives an output)
model.fit(training, output, n_epoch = 1000, batch_size = 8, show_metric = True)

# save the model
model.save("model.tflearn")


Training Step: 1999  | total loss: 0.01339 | time: 0.002s
| Adam | epoch: 1000 | loss: 0.01339 - acc: 0.9996 -- iter: 08/15
Training Step: 2000  | total loss: 0.01296 | time: 0.005s
| Adam | epoch: 1000 | loss: 0.01296 - acc: 0.9997 -- iter: 15/15
--


> Model is working VERY well given the overly simple data that it's seen so far. 99.97% accuracy!  We can expect this to go down when I expand on the intents file.